# Template argument deduction

* Part I: Type Inference in C++ (auto and decltype)
* Part II: Template argument deduction

# Part I: Type Inference in C++ (auto and decltype)

Type Inference refers to automatic deduction of the data type of an expression in a programming language. Before C++ 11, each data type needed to be explicitly declared at compile-time, limiting the values of an expression at runtime but after the new version of C++, many keywords are included which allows a programmer to leave the type deduction to the compiler itself. 
With type inference capabilities, we can spend less time having to write out things the compiler already knows. As all the types are deduced in the compiler phase only, the time for compilation increases slightly but it does not affect the run time of the program.

* auto
* decltype

## auto keyword

1) auto keyword: The auto keyword specifies that the type of the variable that is being declared will be automatically deducted from its initializer. In the case of functions, if their return type is auto then that will be evaluated by return type expression at runtime. Good use of auto is to avoid long initializations when creating iterators for containers.   

> **The variable declared with auto keyword should be initialized at the time of its declaration only or else there will be a compile-time error.** 

In [13]:
#include <bits/stdc++.h>
using namespace std;

In [14]:
// C++ program to demonstrate working of auto
// and type inference
int main(){
	// auto a; this line will give error
	// because 'a' is not initialized at
	// the time of declaration
	// a=33;

	// see here x ,y,ptr are
	// initialised at the time of
	// declaration hence there is
	// no error in them
	auto x = 4;
	auto y = 3.37;
	auto ptr = &x;
	cout << typeid(x).name() << endl
		<< typeid(y).name() << endl
		<< typeid(ptr).name() << endl;

	return 0;
};

In [15]:
main();

i
d
Pi


Typeid is an operator which is used where the dynamic type of an object needs to be known. 

typeid(x).name() returns the data type of x, for example, it return ‘i’ for integers, ‘d’ for doubles, ‘Pi’ for the pointer to integer etc. But the actual name returned is mostly compiler dependent. 

Note: auto becomes int type if even an integer reference is assigned to it. To make it reference type, we use auto &. 

* Function that returns a ‘reference to int’ type : int& fun() {};
* m will default to int type instead of int& type : auto m = fun();
* n will be of int& type because of use of extra & with auto keyword : auto& n = fun();

## decltype Keyword
2) decltype Keyword: It inspects the declared type of an entity or the type of an expression. ‘auto’ lets you declare a variable with a particular type whereas decltype lets you extract the type from the variable so decltype is sort of an operator that evaluates the type of passed expression. 
Explanation of the above keywords and their uses is given below: 

In [18]:
// C++ program to demonstrate use of decltype
// Driver Code
int main()
{
	int x = 5;

	// j will be of type int : data type of x
	decltype(x) j = x + 5;

	cout << typeid(j).name();

	return 0;
};

In [17]:
int var;
const int&& fx();
struct A { double x; };
const A* a = new A();

Next, examine the types that are returned by the four decltype statements in the following table.

* decltype(var);	int	The type of variable var.
* decltype(a->x);	double	The type of the member access.    
* decltype(fx());	const int&&	An rvalue reference to a const int.
* decltype((a->x));	const double&	The inner parentheses cause the statement to be evaluated as an expression instead of a member access. And because a is declared as a const pointer, the type is a reference to const double.

decltype vs typeid

* Decltype gives the type information at compile time while typeid gives at runtime.
* So, if we have a base class reference (or pointer) referring to (or pointing to) a derived class object, the decltype would give type as base class reference (or pointer, but typeid would give the derived type reference (or pointer).

# Part II: Template argument deduction

# 1 CTAD automatically  -- If you do not specify the template parameter explicitly

# motivation

When you call a template function, you may omit any template argument that the compiler can determine or deduce by the usage and context of that template function call.

The compiler tries to deduce a template argument by comparing the type of the corresponding template parameter with the type of the argument used in the function call. The two types that the compiler compares (the template parameter and the argument used in the function call) must be of a certain structure in order for template argument deduction to work. The following lists these type structures:

> motivation1: sometimes you do not need enter the template argument, the compiler will help you deduce it automtically

Starting in C++17, when instantiating an object from a class template, the compiler can deduce the template types from the types of the object’s initializer (this is called class template argument deduction or CTAD for short). For example:

In [2]:
#include <utility> // for std::pair

int main()
{
    std::pair<int, int> p1{ 1, 2 }; // explicitly specify class template std::pair<int, int> (C++11 onward)
    std::pair p2{ 1, 2 };           // CTAD used to deduce std::pair<int, int> from the initializers (C++17)

    return 0;
}

CTAD is only performed if no template argument list is present. Therefore, both of the following are errors:

In [ ]:
#include <utility> // for std::pair

int main()
{
    std::pair<> p1 { 1, 2 };    // error: too few template arguments, both arguments not deduced
    std::pair<int> p2 { 3, 4 }; // error: too few template arguments, second argument not deduced

    return 0;
}

# 2 deduction guides -if automatical CTAD  failed

In general, CTAD automatically works when class templates have constructors whose signatures mention all of the class template parameters (like MyPair above). However, sometimes constructors themselves are templated, which breaks the connection that CTAD relies on. In those cases, the author of the class template can provide “deduction guides” that tell the compiler how to deduce class template arguments from constructor arguments.

In most cases, CTAD works right out of the box. However, in certain cases, the compiler may need a little extra help understanding how to deduce the template arguments properly.

> motivation2: sometimes you have to provide “deduction guides” that tell the compiler how to deduce class template arguments. 

You may be surprised to find that the following program (which is almost identical to the example that uses std::pair above) doesn’t compile in C++17:

In [3]:
// define our own pair type
template <typename T, typename U>
struct pair
{
    T first{};
    U second{};
};

int main()
{
    pair<int, int> p1{ 1, 2 }; // ok: we're explicitly specifying the template arguments
    pair p2{ 1, 2 };           // compile error in C++17

    return 0;
}

input_line_10:11:10: error: no viable constructor or deduction guide for deduction of template arguments of
      'pair'
    pair p2{ 1, 2 };           // compile error in C++17
         ^
input_line_10:3:8: note: candidate function template not viable: requires 0 arguments, but 2 were
      provided
struct pair
       ^
input_line_10:3:8: note: candidate function template not viable: requires 1 argument, but 2 were provided


Interpreter Error: 

If you compile this in C++17, you’ll likely get some error about “class template argument deduction failed” or “cannot deduce template arguments” or “No viable constructor or deduction guide”. This is because in C++17, CTAD doesn’t know how to deduce the template arguments for aggregate class templates. To address this, we can provide the compiler with a deduction guide, which tells the compiler how to deduce the template arguments for a given class template.

Here’s the same program with a deduction guide:

In [5]:
template <typename T, typename U>
struct pair
{
    T first{};
    U second{};
};

// Here's a deduction guide for our pair class
// pair objects initialized with arguments of type T and U should deduce to pair<T, U>
template <typename T, typename U>
pair(T, U) -> pair<T, U>;

int main()
{
    pair<int, int> p1{ 1, 2 }; // explicitly specify class template pair<int, int> (C++11 onward)
    pair p2{ 1, 2 };     // CTAD used to deduce pair<int, int> from the initializers (C++17)

    return 0;
};

input_line_12:10:1: error: deduction guide must be declared in the same scope as template
      '__cling_N56::pair'
pair(T, U) -> pair<T, U>;
^
input_line_12:2:8: note: template is declared here
struct pair
       ^
input_line_12:14:10: error: no viable constructor or deduction guide for deduction of template arguments of
      'pair'
    pair p2{ 1, 2 };     // CTAD used to deduce pair<int, int> from the ...
         ^
input_line_12:2:8: note: candidate function template not viable: requires 0 arguments, but 2 were
      provided
struct pair
       ^
input_line_12:2:8: note: candidate function template not viable: requires 1 argument, but 2 were provided


Interpreter Error: 

This example should compile under C++17.

The deduction guide for our pair class is pretty simple, but let’s take a closer look at how it works.

In [ ]:
// Here's a deduction guide for our pair class
// pair objects initialized with arguments of type T and U should deduce to pair<T, U>
template <typename T, typename U>
pair(T, U) -> pair<T, U>;

First, we use the same template type definition as in our pair class. This makes sense, because if our deduction guide is going to tell the compiler how to deduce the types for a pair<T, U>, we have to define what T and U are (template types). Second, on the right hand side of the arrow, we have the type that we’re helping the compiler to deduce. In this case, we want the compiler to be able to deduce template arguments for objects of type pair<T, U>, so that’s exactly what we put here. Finally, on the left side of the arrow, we tell the compiler what kind of declaration to look for. In this case, we’re telling it to look for a declaration of some object named pair with two arguments (one of type T, the other of type U). We could also write this as pair(T t, U u) (where t and u are the names of the parameters, but since we don’t use t and u, we don’t need to give them names).

Putting it all together, we’re telling the compiler that if it sees a declaration of a pair with two arguments (of types T and U respectively), it should deduce the type to be a pair<T, U>.

So when the compiler sees the definition pair p2{ 1, 2 }; in our program, it will say, “oh, this is a declaration of a pair and there are two arguments of type int and int, so using the deduction guide, I should deduce this to be a pair<int, int>“.

# 3 how to deduce the type of template argument  if 
(1) omit any template argument

(2) function arguments depends on the types of its template arguments

(3) the type of function arguments and passed parameter might vary because of const and & keyword (implicit conversition)

> mitivation3: you need to know the deduced type, because it affect your operation in your function and output

In [ ]:
template<typename T>
void f(T& param){ //param is a reference
    T a_variable; // what the type of the a_variable at different senario?
};      

const int cx = x;      // cx is a const int

f(x);  ///???

Template Generic Syntax
~~~
template<typename T>
void f(ParamType param);

f(expr);
~~~

**Case 1: ParamType is a Reference or Pointer, but not a Universal or Forward Reference.** In this case type deduction works this way.
* The compiler ignores the reference part if it exists in expr. 
* The compiler then pattern-matches expr's type against ParamType to determing T.

In [ ]:
template<typename T>
void f(T& param){ //param is a reference
    T a_variable; // what the type of the a_variable at different senario?
};      

int x = 27;            // x is an int
const int cx = x;      // cx is a const int
const int& rx = x;     // rx is a reference to x as a const int

f(x);                  // T is int, param's type is int&
f(cx);                 // T is const int, param's type is const int&
f(rx);                 // T is const int, param's type is const int&

In [ ]:
* step1: according to the function argument and parameter with implicite conversition principle, determine the paramType
~~~

~~~
* step2: according the the template argument and  paramType, determine the T

**Case 2: ParamType is a Universal Reference or Forward Reference.** 
* In this case type deduction is the same as in case 1 if the expr is an rvalue. 
* If expr is an lvalue, both T and ParamType are deduced to be lvalue references.

In [ ]:
template<typename T>
void f(T&& param);     // param is a universal reference

int x = 27;            // x is an int
const int cx = x;      // cx is a const int
const int& rx = x;     // rx is a reference to x as a const int

f(x);                  // x is lvalue, so T is int&, param's type is also int&
f(cx);                 // cx is lvalue, so T is const int&, param's type is also const int&
f(rx);                 // rx is lvalue, so T is const int&, param's type is also const int&
f(27);                 // 27 is rvalue, so T is int, param's type is therefore int&&

Case 3: ParamType is Neither a Pointer nor a Reference. 
* If expr is a reference the reference part is ignored. 
* If expr is const that is ignored as well. 
* If it is volatile that is also ignored when deducing T's type.

In [ ]:
template<typename T>
void f(T param);       // param is now passed by value

int x = 27;            // x is an int
const int cx = x;      // cx is a const int
const int& rx = x;     // rx is a reference to x as a const int

f(x);                  // T's and param's types are both int
f(cx);                 // T's and param's types are again both int
f(rx);                 // T's and param's types are still both int

# 4 how declare a template function whose return type depends on the types of its template arguments.

The decltype type specifier yields the type of a specified expression. The decltype type specifier, together with the auto keyword, is useful primarily to developers who write template libraries. Use auto and decltype to declare a template function whose return type depends on the types of its template arguments. Or, use auto and decltype to declare a template function that wraps a call to another function, and then returns the return type of the wrapped function.

In C++14, you can use decltype(auto) with no trailing return type to declare a template function whose return type depends on the types of its template arguments.

In C++11, you can use the decltype type specifier on a trailing return type, together with the auto keyword, to declare a template function whose return type depends on the types of its template arguments. For example, consider the following code example in which the return type of the template function depends on the types of the template arguments. In the code example, the UNKNOWN placeholder indicates that the return type cannot be specified.

In [ ]:
template<typename T, typename U>
UNKNOWN func(T&& t, U&& u){ return t + u; };

The introduction of the decltype type specifier enables a developer to obtain the type of the expression that the template function returns. Use the alternative function declaration syntax that is shown later, the auto keyword, and the decltype type specifier to declare a late-specified return type. The late-specified return type is determined when the declaration is compiled, instead of when it is coded.

In the following code example, the late-specified return type of the myFunc template function is determined by the types of the t and u template arguments. As a best coding practice, the code example also uses rvalue references and the forward function template, which support perfect forwarding. For more information, see Rvalue Reference Declarator: &&.

In [ ]:
//C++11
template<typename T, typename U>
auto myFunc(T&& t, U&& u) -> decltype (forward<T>(t) + forward<U>(u))
        { return forward<T>(t) + forward<U>(u); };

//C++14
template<typename T, typename U>
decltype(auto) myFunc(T&& t, U&& u)
        { return forward<T>(t) + forward<U>(u); };

# reference
* https://riptutorial.com/cplusplus/example/25566/template-type-deduction